# Cluster and Segmentation Best Hotel in Rome

## 1. Introduction

When someone travel to work, do tourist, visit another person,etc. One of the most important decision is to choose the place where he/she  will stay.
If is a business travel maybe the best choice is a  place near to work and some specific venues, if is to do tourist maybe the best choice is a place near to tour agencies or restaurants. In this case I will focus in hotels in Rome

## 2. Business Problem

Let assume that you work in a travel agency and one day a client come to your services, because he want travel to Rome,Italy to do tourist and need information about hotels near to restaurants, tourist agency and more places to enjoy his journey. which are the best groups of hotels in Rome,Italy that you could recommend to your client according his specifications?

## 3. Data 

Number of existing facilities and their type and location in every hotels will be obtained using Foursquare API. 

### 3.1 Explore the data

import the neccesary libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
from random import seed
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
print('folium imported')
#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
print('nominatim imported')
print('Libraries imported.')

folium imported
nominatim imported
Libraries imported.


## Define Foursquare Credentials and Version

In [2]:
CLIENT_ID = 'VBK130EFHFXH3R5ZXE5QBDBOOMIF5M0DY4MUDFXRO3ACL5F1' # your Foursquare ID
CLIENT_SECRET = 'G0BMGCDKIDMDUSSHVN23VI405YYDZEVR1ONDA1WEENRJELKA' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


Your credentails:
CLIENT_ID: VBK130EFHFXH3R5ZXE5QBDBOOMIF5M0DY4MUDFXRO3ACL5F1
CLIENT_SECRET:G0BMGCDKIDMDUSSHVN23VI405YYDZEVR1ONDA1WEENRJELKA


### We are located in Rome

In [3]:
address = 'Rome, Italy'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

41.894802 12.4853384


### Searching hotels around Rome

In [4]:
search_query = 'Hotel'
radius = 1300
print(search_query + ' .... OK!')

Hotel .... OK!


In [5]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=VBK130EFHFXH3R5ZXE5QBDBOOMIF5M0DY4MUDFXRO3ACL5F1&client_secret=G0BMGCDKIDMDUSSHVN23VI405YYDZEVR1ONDA1WEENRJELKA&ll=41.894802,12.4853384&v=20180604&query=Hotel&radius=1300&limit=100'

### Results of hotels in Rome

In [6]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c8f04cf9fb6b753b45439e6'},
 'response': {'venues': [{'id': '4ba5f478f964a520182b39e3',
    'name': 'Hotel Forum',
    'location': {'address': "Via Tor De' Conti, 25",
     'lat': 41.89384,
     'lng': 12.48773,
     'labeledLatLngs': [{'label': 'display',
       'lat': 41.89384,
       'lng': 12.48773}],
     'distance': 225,
     'postalCode': '00184',
     'cc': 'IT',
     'city': 'Roma',
     'state': 'Lazio',
     'country': 'Italia',
     'formattedAddress': ["Via Tor De' Conti, 25",
      '00184 Roma Lazio',
      'Italia']},
    'categories': [{'id': '4bf58dd8d48988d1fa931735',
      'name': 'Hotel',
      'pluralName': 'Hotels',
      'shortName': 'Hotel',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/hotel_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1552876751',
    'hasPerk': False},
   {'id': '4adcdabff964a520735121e3',
    'name': 'Hotel Pace Helvezia',
    'location': {'address':

In [7]:
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
print(dataframe.shape)
dataframe.head()

(50, 19)


,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",False,4ba5f478f964a520182b39e3,"Via Tor De' Conti, 25",IT,Roma,Italia,NaN,225,"[Via Tor De' Conti, 25, 00184 Roma Lazio, Italia]","[{'label': 'display', 'lat': 41.89384, 'lng': ...",41.893840,12.487730,NaN,00184,Lazio,Hotel Forum,v-1552876751,NaN
1,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",False,4adcdabff964a520735121e3,Via 4 Novembre 104,IT,Roma,Italia,NaN,233,"[Via 4 Novembre 104, 00187 Roma Lazio, Italia]","[{'label': 'display', 'lat': 41.89688683175500...",41.896887,12.484984,NaN,00187,Lazio,Hotel Pace Helvezia,v-1552876751,NaN
2,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",False,4bc889c3dc55eee1a389e8ac,Via dei Serpenti 109,IT,Roma,Italia,NaN,447,"[Via dei Serpenti 109, 00184 Roma Lazio, Italia]","[{'label': 'display', 'lat': 41.89670113293939...",41.896701,12.490097,NaN,00184,Lazio,Hotel Apollo,v-1552876751,NaN
3,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",False,4ce6810e8ef78cfa34cb8c9b,"Via Nazionale, 66",IT,Roma,Italia,NaN,716,"[Via Nazionale, 66, 00187 Roma Lazio, Italia]","[{'label': 'display', 'lat': 41.89932835379613...",41.899328,12.491482,NaN,00187,Lazio,Hotel Esposizione,v-1552876751,NaN
4,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",False,4adcdabef964a5200c5121e3,Piazza Trinità dei Monti 6,IT,Roma,Italia,NaN,1245,"[Piazza Trinità dei Monti 6, 00187 Roma Lazio,...","[{'label': 'display', 'lat': 41.90592983350055...",41.905930,12.483732,NaN,00187,Lazio,Hotel Hassler Roma,v-1552876751,NaN


### Creating a dataframe filtered

In [8]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Hotel Forum,Hotel,"Via Tor De' Conti, 25",IT,Roma,Italia,NaN,225,"[Via Tor De' Conti, 25, 00184 Roma Lazio, Italia]","[{'label': 'display', 'lat': 41.89384, 'lng': ...",41.893840,12.487730,NaN,00184,Lazio,4ba5f478f964a520182b39e3
1,Hotel Pace Helvezia,Hotel,Via 4 Novembre 104,IT,Roma,Italia,NaN,233,"[Via 4 Novembre 104, 00187 Roma Lazio, Italia]","[{'label': 'display', 'lat': 41.89688683175500...",41.896887,12.484984,NaN,00187,Lazio,4adcdabff964a520735121e3
2,Hotel Apollo,Hotel,Via dei Serpenti 109,IT,Roma,Italia,NaN,447,"[Via dei Serpenti 109, 00184 Roma Lazio, Italia]","[{'label': 'display', 'lat': 41.89670113293939...",41.896701,12.490097,NaN,00184,Lazio,4bc889c3dc55eee1a389e8ac
3,Hotel Esposizione,Hotel,"Via Nazionale, 66",IT,Roma,Italia,NaN,716,"[Via Nazionale, 66, 00187 Roma Lazio, Italia]","[{'label': 'display', 'lat': 41.89932835379613...",41.899328,12.491482,NaN,00187,Lazio,4ce6810e8ef78cfa34cb8c9b
4,Hotel Hassler Roma,Hotel,Piazza Trinità dei Monti 6,IT,Roma,Italia,NaN,1245,"[Piazza Trinità dei Monti 6, 00187 Roma Lazio,...","[{'label': 'display', 'lat': 41.90592983350055...",41.905930,12.483732,NaN,00187,Lazio,4adcdabef964a5200c5121e3
5,Hotel Raffaello,Hotel,Via Urbana 3,IT,Roma,Italia,NaN,913,"[Via Urbana 3, 00184 Roma Lazio, Italia]","[{'label': 'display', 'lat': 41.89776997207932...",41.897770,12.495622,NaN,00184,Lazio,4bc88aebdc55eee1808ae8ac
6,Hotel Galatea,Hotel,Via Genova 24,IT,Roma,Italia,NaN,794,"[Via Genova 24, Roma Lazio, Italia]","[{'label': 'display', 'lat': 41.8994855764514,...",41.899486,12.492568,NaN,NaN,Lazio,4bcc607faeaaeee10bf53d6d
7,Hotel Diana,Hotel,Via Principe Amedeo 4,IT,Roma,Italia,NaN,1193,"[Via Principe Amedeo 4, 00185 Roma Lazio, Italia]","[{'label': 'display', 'lat': 41.90065681639322...",41.900657,12.497402,NaN,00185,Lazio,4adcdabef964a5201c5121e3
8,Hotel Giolli,Hotel,Via Nazionale 69,IT,Roma,Italia,NaN,696,"[Via Nazionale 69, 00184 Roma Lazio, Italia]","[{'label': 'display', 'lat': 41.89928657123711...",41.899287,12.491204,NaN,00184,Lazio,4dc9cfafd4c07b3501083f57
9,Hotel Fori Imperiali Cavalieri,Hotel,Via Frangipane 34,IT,Roma,Italia,NaN,387,"[Via Frangipane 34, 00184 Roma Lazio, Italia]","[{'label': 'display', 'lat': 41.89316511063772...",41.893165,12.489464,NaN,00184,Lazio,4c0c1a17a1b32d7f830f9cf0


### glance of the hotels

In [10]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Rome',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

# 4. Metholodogy 

In this project we will direct our efforts on detecting venues around the hotels. We will limit our analysis to area 500 meters around each hotel.

In first step we have collected the required **data: location and type (category) of every venue within 500 meters around each hotel**,we will use **maps** to visualize the hotels in Rome.

Second step in our analysis will be calculation and exploration of '**venues**' across different areas of hootels-

In third and final step we will focus on most promising areas and within those create **clusters of locations that meet the requirements** established in discussion with stakeholders.

## Analysis

In [11]:
df=dataframe_filtered[['name','categories','lat','lng']]
print(df.shape)
df.head()

(50, 4)


,name,categories,lat,lng
0,Hotel Forum,Hotel,41.893840,12.487730
1,Hotel Pace Helvezia,Hotel,41.896887,12.484984
2,Hotel Apollo,Hotel,41.896701,12.490097
3,Hotel Esposizione,Hotel,41.899328,12.491482
4,Hotel Hassler Roma,Hotel,41.905930,12.483732


### Define functions
The first function is to get the names of categories and the second is to get all the venues in a radius of 500 meters

In [12]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [13]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['name', 
                  'Latitude', 
                  'Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

**Get all venues aroud each hotel**

In [14]:
hotels_venues = getNearbyVenues(names=df['name'],
                                   latitudes=df['lat'],
                                   longitudes=df['lng']
                                  )
print(hotels_venues.shape)

Hotel Forum
Hotel Pace Helvezia
Hotel Apollo
Hotel Esposizione
Hotel Hassler Roma
Hotel Raffaello
Hotel Galatea
Hotel Diana
Hotel Giolli
Hotel Fori Imperiali Cavalieri
Hotel Mediterraneo
The Fifteen Keys Hotel
iQ Hotel Roma
Hotel Capo D'Africa Rome
Hotel Ambasciatori Palace
Hotel Sonya
Welcome Piram Hotel Rome
Hotel Rex Rome
Hotel Concordia
Grand Hotel Palace
Hotel Campo De' Fiori
Hotel Art
Hotel Sistina - Roma
939 Hotel (Hotel Marcus)
Hotel Gladiatori Palazzo Manfredi
Hotel D'Este
Baglioni Hotel Regina
Hotel San Anselmo
Downtown Accommodation Hotel
Hotel Amalfi Rome
Hotel Miami Rome
Hotel Diocleziano
Hotel Cecil Rome
Hotel Concorde
Hotel Arenula
Hotel De Monti
Raphaël Hotel
Hotel Cosmopolita Rome
La Griffe Hotel Rome
Harry's Bar Trevi Hotel & Restaurant
Edera Hotel
Hotel Saturnia
Hotel Scalinata di Spagna
Hotel Domus Romana
B&B Trevi Hotel
Hotel Alius
Hotel Swisse
Hotel Martini
Hotel Traiano Roma
Roma Luxus Hotel
(4659, 7)


**Exploring the new dataframe**

In [15]:

dele=hotels_venues[(hotels_venues['Venue Category'] == 'Hotel') | (hotels_venues['Venue Category'] == 'Hostel')].index
hotels_venues=hotels_venues.drop(dele,axis=0).reset_index(drop=True)
print(hotels_venues.shape)

(3797, 7)


**Counting all the venues around each hotel**

In [16]:
hotels_venues.groupby('name').count()

,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
name,,,,,,
939 Hotel (Hotel Marcus),97,97,97,97,97,97
B&B Trevi Hotel,95,95,95,95,95,95
Baglioni Hotel Regina,69,69,69,69,69,69
Downtown Accommodation Hotel,92,92,92,92,92,92
Edera Hotel,48,48,48,48,48,48
Grand Hotel Palace,70,70,70,70,70,70
Harry's Bar Trevi Hotel & Restaurant,95,95,95,95,95,95
Hotel Alius,57,57,57,57,57,57
Hotel Amalfi Rome,80,80,80,80,80,80


In [17]:
print('There are {} uniques categories.'.format(len(hotels_venues['Venue Category'].unique())))

There are 135 uniques categories.


### Transform categorical variables to indicator variables

In [18]:
hotels_onehot = pd.get_dummies(hotels_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
hotels_onehot['name'] = hotels_venues['name'] 

# move neighborhood column to the first column
fixed_columns = [hotels_onehot.columns[-1]] + list(hotels_onehot.columns[:-1])
hotels_onehot = hotels_onehot[fixed_columns]

print('shape of hotels_onehot:',hotels_onehot.shape)
hotels_onehot.head()

shape of hotels_onehot: (3797, 136)


,name,Abruzzo Restaurant,Accessories Store,American Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Bakery,Bar,Bed & Breakfast,Beer Bar,Beer Store,Bistro,Boarding House,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Building,Café,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Cosmetics Shop,Courthouse,Creperie,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Electronics Store,Embassy / Consulate,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Furniture / Home Store,Garden,Gastropub,Gay Bar,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Gym,Gym / Fitness Center,Halal Restaurant,Health Food Store,Historic Site,History Museum,Hotel Bar,Ice Cream Shop,Indian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kebab Restaurant,Lingerie Store,Lounge,Market,Mediterranean Restaurant,Memorial Site,Men's Store,Middle Eastern Restaurant,Monument / Landmark,Movie Theater,Multiplex,Museum,Nightclub,Opera House,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pastry Shop,Peruvian Restaurant,Piano Bar,Pie Shop,Pizza Place,Plaza,Pub,Public Art,Record Shop,Recording Studio,Resort,Restaurant,Road,Roman Restaurant,Salad Place,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Snack Place,South American Restaurant,Souvenir Shop,Spa,Speakeasy,Sporting Goods Shop,Supermarket,Szechuan Restaurant,Tea Room,Temple,Theater,Thrift / Vintage Store,Tour Provider,Toy / Game Store,Trattoria/Osteria,Tuscan Restaurant,Vegetarian / Vegan Restaurant,Wine Bar,Wine Shop,Winery,Women's Store
0,Hotel Forum,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Hotel Forum,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,Hotel Forum,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,Hotel Forum,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Hotel Forum,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


**Here we can find the frequencies of each venue in each hotels**

In [19]:

hotels_grouped = hotels_onehot.groupby('name').mean().reset_index()
print('shape of hotels_grouped:',hotels_grouped.shape)
hotels_grouped.head()

shape of hotels_grouped: (50, 136)


,name,Abruzzo Restaurant,Accessories Store,American Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Bakery,Bar,Bed & Breakfast,Beer Bar,Beer Store,Bistro,Boarding House,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Building,Café,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Cosmetics Shop,Courthouse,Creperie,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Electronics Store,Embassy / Consulate,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Furniture / Home Store,Garden,Gastropub,Gay Bar,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Gym,Gym / Fitness Center,Halal Restaurant,Health Food Store,Historic Site,History Museum,Hotel Bar,Ice Cream Shop,Indian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kebab Restaurant,Lingerie Store,Lounge,Market,Mediterranean Restaurant,Memorial Site,Men's Store,Middle Eastern Restaurant,Monument / Landmark,Movie Theater,Multiplex,Museum,Nightclub,Opera House,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pastry Shop,Peruvian Restaurant,Piano Bar,Pie Shop,Pizza Place,Plaza,Pub,Public Art,Record Shop,Recording Studio,Resort,Restaurant,Road,Roman Restaurant,Salad Place,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Snack Place,South American Restaurant,Souvenir Shop,Spa,Speakeasy,Sporting Goods Shop,Supermarket,Szechuan Restaurant,Tea Room,Temple,Theater,Thrift / Vintage Store,Tour Provider,Toy / Game Store,Trattoria/Osteria,Tuscan Restaurant,Vegetarian / Vegan Restaurant,Wine Bar,Wine Shop,Winery,Women's Store
0,939 Hotel (Hotel Marcus),0.0,0.000000,0.000000,0.0,0.000000,0.030928,0.0,0.0,0.010309,0.000000,0.000000,0.00000,0.000000,0.030928,0.000000,0.000000,0.041237,0.000000,0.0,0.000000,0.010309,0.000000,0.020619,0.0,0.010309,0.010309,0.000000,0.020619,0.010309,0.010309,0.000000,0.010309,0.010309,0.0,0.0,0.0,0.0,0.000000,0.010309,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.030928,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.00000,0.0,0.082474,0.000000,0.0,0.206186,0.010309,0.000000,0.010309,0.010309,0.0,0.0,0.010309,0.0,0.000000,0.0,0.0,0.0,0.030928,0.000000,0.0,0.000000,0.010309,0.0,0.00000,0.0,0.00000,0.00000,0.0,0.0,0.010309,0.051546,0.103093,0.000000,0.00000,0.000000,0.0,0.0,0.030928,0.010309,0.010309,0.00000,0.030928,0.0,0.0,0.010309,0.0,0.0,0.010309,0.010309,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.030928,0.041237,0.0,0.0,0.000000,0.0,0.0,0.010309
1,B&B Trevi Hotel,0.0,0.000000,0.000000,0.0,0.010526,0.031579,0.0,0.0,0.000000,0.010526,0.031579,0.00000,0.000000,0.000000,0.010526,0.010526,0.000000,0.010526,0.0,0.010526,0.000000,0.000000,0.031579,0.0,0.000000,0.000000,0.010526,0.021053,0.000000,0.000000,0.000000,0.010526,0.000000,0.0,0.0,0.0,0.0,0.021053,0.000000,0.000000,0.0,0.0,0.0,0.010526,0.0,0.0,0.0,0.000000,0.0,0.0,0.031579,0.000000,0.0,0.010526,0.0,0.000000,0.0,0.021053,0.0,0.0,0.000000,0.0,0.0,0.0,0.021053,0.00000,0.0,0.084211,0.000000,0.0,0.200000,0.010526,0.010526,0.010526,0.010526,0.0,0.0,0.010526,0.0,0.010526,0.0,0.0,0.0,0.031579,0.000000,0.0,0.010526,0.000000,0.0,0.00000,0.0,0.00000,0.00000,0.0,0.0,0.000000,0.031579,0.073684,0.010526,0.00000,0.000000,0.0,0.0,0.031579,0.000000,0.010526,0.00000,0.031579,0.0,0.0,0.000000,0.0,0.0,0.021053,0.010526,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.010526,0.000000,0.010526,0.00000,0.000000,0.031579,0.021053,0.0,0.0,0.000000,0.0,0.0,0.000000
2,Baglioni Hotel Regina,0.0,0.000000,0.014493,0.0,0.000000,0.014493,0.0,0.0,0.000000,0.014493,0.014493,0.00000,0.000000,0.057971,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.028986,0.0,0.014493

**See the hotels with the frequencies**

In [20]:
num_top_venues = 5

for hood in hotels_grouped['name']:
    print("----"+hood+"----")
    temp = hotels_grouped[hotels_grouped['name'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----939 Hotel (Hotel Marcus)----
                venue  freq
0  Italian Restaurant  0.21
1               Plaza  0.10
2      Ice Cream Shop  0.08
3         Pizza Place  0.05
4            Boutique  0.04


----B&B Trevi Hotel----
                 venue  freq
0   Italian Restaurant  0.20
1       Ice Cream Shop  0.08
2                Plaza  0.07
3          Pizza Place  0.03
4  Monument / Landmark  0.03


----Baglioni Hotel Regina----
                venue  freq
0  Italian Restaurant  0.25
1          Restaurant  0.06
2   Trattoria/Osteria  0.06
3              Bistro  0.06
4         Pizza Place  0.04


----Downtown Accommodation Hotel----
                venue  freq
0  Italian Restaurant  0.25
1       Historic Site  0.13
2      Ice Cream Shop  0.04
3         Pizza Place  0.04
4            Wine Bar  0.04


----Edera Hotel----
                venue  freq
0  Italian Restaurant  0.21
1                Café  0.10
2         Pizza Place  0.08
3        Cocktail Bar  0.04
4   French Restaurant  0.04




This function return the 'n' first venues

In [21]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### Make a Dataframe of each hotels with their 10 top venues

In [22]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['name']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
names_venues_sorted = pd.DataFrame(columns=columns)
names_venues_sorted['name'] = hotels_grouped['name']

for ind in np.arange(hotels_grouped.shape[0]):
    names_venues_sorted.iloc[ind, 1:] = return_most_common_venues(hotels_grouped.iloc[ind, :], num_top_venues)

names_venues_sorted.head()

,name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,939 Hotel (Hotel Marcus),Italian Restaurant,Plaza,Ice Cream Shop,Pizza Place,Boutique,Trattoria/Osteria,Bistro,Toy / Game Store,Sandwich Place,Art Museum
1,B&B Trevi Hotel,Italian Restaurant,Ice Cream Shop,Plaza,Sandwich Place,Pizza Place,Monument / Landmark,Bed & Breakfast,Restaurant,Café,Toy / Game Store
2,Baglioni Hotel Regina,Italian Restaurant,Trattoria/Osteria,Restaurant,Bistro,Pizza Place,Cocktail Bar,Movie Theater,Café,Gym,Fountain
3,Downtown Accommodation Hotel,Italian Restaurant,Historic Site,Ice Cream Shop,Pizza Place,Sandwich Place,Wine Bar,Monument / Landmark,Temple,Plaza,Indian Restaurant
4,Edera Hotel,Italian Restaurant,Café,Pizza Place,Lounge,Wine Bar,Cocktail Bar,Trattoria/Osteria,Bakery,French Restaurant,Ice Cream Shop


### Starting the K-means segmentation

In [23]:
kclusters = 4
seed(27)
hotels_grouped_clustering = hotels_grouped.drop('name', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(hotels_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 2, 3, 1, 3, 3, 2, 0, 3, 3], dtype=int32)

**Inserting cluster labels to the dataframe**

In [24]:
# add clustering labels
names_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

hotels_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
hotels_merged = hotels_merged.join(names_venues_sorted.set_index('name'), on='name')

hotels_merged.head() # check the last columns!

,name,categories,lat,lng,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Hotel Forum,Hotel,41.893840,12.487730,1,Italian Restaurant,Historic Site,Temple,History Museum,Ice Cream Shop,Pizza Place,Plaza,Sandwich Place,Monument / Landmark,Wine Bar
1,Hotel Pace Helvezia,Hotel,41.896887,12.484984,1,Italian Restaurant,Historic Site,Ice Cream Shop,Wine Bar,History Museum,Art Museum,Plaza,Restaurant,Monument / Landmark,Museum
2,Hotel Apollo,Hotel,41.896701,12.490097,0,Italian Restaurant,Ice Cream Shop,Pizza Place,Wine Bar,Historic Site,Coffee Shop,Sandwich Place,Bed & Breakfast,Indian Restaurant,Plaza
3,Hotel Esposizione,Hotel,41.899328,12.491482,0,Italian Restaurant,Ice Cream Shop,Pizza Place,Wine Bar,Bed & Breakfast,Cocktail Bar,Roman Restaurant,Sandwich Place,Plaza,Art Museum
4,Hotel Hassler Roma,Hotel,41.905930,12.483732,2,Italian Restaurant,Boutique,Café,Plaza,Pizza Place,Jewelry Store,Japanese Restaurant,Trattoria/Osteria,Ice Cream Shop,Tea Room


In [25]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(hotels_merged['lat'], hotels_merged['lng'],hotels_merged['name'], hotels_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [57]:
k1=hotels_merged.loc[hotels_merged['Cluster Labels'] == 0, hotels_merged.columns[[1] + list(range(5, hotels_merged.shape[1]))]]
display(k1)
display(hotels_merged.iloc[k1.index,0].tolist())
k1.shape

,categories,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Hotel,Italian Restaurant,Ice Cream Shop,Pizza Place,Wine Bar,Coffee Shop,Sandwich Place,Historic Site,Bed & Breakfast,Indian Restaurant,Cocktail Bar
3,Hotel,Italian Restaurant,Ice Cream Shop,Pizza Place,Wine Bar,Bed & Breakfast,Cocktail Bar,Café,Plaza,Sandwich Place,Park
5,Hotel,Italian Restaurant,Ice Cream Shop,Pizza Place,Wine Bar,Cocktail Bar,Bed & Breakfast,Sandwich Place,Roman Restaurant,Plaza,Art Museum
6,Hotel,Italian Restaurant,Ice Cream Shop,Pizza Place,Wine Bar,Cocktail Bar,Bed & Breakfast,Plaza,Art Museum,History Museum,Bistro
7,Hotel,Italian Restaurant,Ice Cream Shop,Pizza Place,Wine Bar,Cocktail Bar,Bed & Breakfast,Roman Restaurant,Art Museum,Sandwich Place,Plaza
11,Hotel,Italian Restaurant,Ice Cream Shop,Café,Bed & Breakfast,Cocktail Bar,Plaza,Pizza Place,Bistro,History Museum,Fountain
12,Hotel,Italian Restaurant,Ice Cream Shop,Pizza Place,Wine Bar,Bed & Breakfast,Plaza,Café,Cocktail Bar,Park,Sandwich Place
13,Hotel,Italian Restaurant,Bed & Breakfast,Café,Ice Cream Shop,Plaza,Cocktail Bar,Pizza Place,Bistro,Fountain,History Museum
27,Hotel,Italian Restaurant,Ice Cream Shop,Café,History Museum,Bed & Breakfast,Restaurant,Cocktail Bar,Pizza Place,Plaza,Movie Theater
33,Hotel,Italian Restaurant,Pizza Place,Ice Cream Shop,Sandwich Place,Plaza,Fountain,Art Museum,Restaurant,Café,Cocktail Bar


['Hotel Apollo',
 'Hotel Raffaello',
 'Hotel Esposizione',
 'Hotel Galatea',
 'Hotel Giolli',
 'iQ Hotel Roma',
 'The Fifteen Keys Hotel',
 'Hotel Sonya',
 'Hotel Miami Rome',
 'Raphaël Hotel',
 'Hotel De Monti',
 'La Griffe Hotel Rome',
 'Hotel Saturnia',
 'Hotel Domus Romana',
 'Hotel Alius',
 'Hotel Martini']

(16, 11)

In [58]:
k2=hotels_merged.loc[hotels_merged['Cluster Labels'] == 1, hotels_merged.columns[[1] + list(range(5, hotels_merged.shape[1]))]]
display(k2)
display(hotels_merged.iloc[k2.index,0].tolist())
k2.shape

,categories,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Hotel,Italian Restaurant,Boutique,Café,Plaza,Jewelry Store,Pizza Place,Trattoria/Osteria,Japanese Restaurant,Road,Restaurant
16,Hotel,Italian Restaurant,Restaurant,Pizza Place,Trattoria/Osteria,Gym,Café,Bed & Breakfast,Cocktail Bar,Middle Eastern Restaurant,Fountain
18,Hotel,Italian Restaurant,Trattoria/Osteria,Bistro,Restaurant,Pizza Place,Cocktail Bar,Movie Theater,Gym,Plaza,Café
19,Hotel,Italian Restaurant,Boutique,Ice Cream Shop,Plaza,Café,Pizza Place,Restaurant,Trattoria/Osteria,Fountain,Bed & Breakfast
20,Hotel,Italian Restaurant,Plaza,Pizza Place,Sandwich Place,Wine Bar,Fountain,Café,Ice Cream Shop,Seafood Restaurant,Dessert Shop
21,Bed & Breakfast,Italian Restaurant,Ice Cream Shop,Boutique,Plaza,Bed & Breakfast,Restaurant,Fountain,Trattoria/Osteria,Toy / Game Store,Pizza Place
23,Hotel,Italian Restaurant,Plaza,Ice Cream Shop,Pizza Place,Trattoria/Osteria,Boutique,Restaurant,Fountain,Sandwich Place,Toy / Game Store
24,Hotel,Italian Restaurant,Plaza,Park,Ice Cream Shop,Gastropub,Roman Restaurant,Scenic Lookout,Café,Creperie,Pizza Place
25,Hotel,Italian Restaurant,Boutique,Plaza,Ice Cream Shop,Historic Site,Pizza Place,Cosmetics Shop,Jewelry Store,Mediterranean Restaurant,Road
28,Hotel,Italian Restaurant,Trattoria/Osteria,Bistro,Restaurant,Pizza Place,Cocktail Bar,Movie Theater,Gym,Plaza,Café


['Hotel Hassler Roma',
 'Hotel Ambasciatori Palace',
 'Grand Hotel Palace',
 'Hotel Concordia',
 "Hotel Campo De' Fiori",
 'Hotel Sistina - Roma',
 '939 Hotel (Hotel Marcus)',
 'Hotel San Anselmo',
 'Hotel Art',
 'Baglioni Hotel Regina',
 'Hotel Cecil Rome',
 "Harry's Bar Trevi Hotel & Restaurant",
 'Hotel Arenula',
 'Hotel Scalinata di Spagna',
 'B&B Trevi Hotel',
 'Tiziano Hotel']

(16, 11)

In [59]:
k3=hotels_merged.loc[hotels_merged['Cluster Labels'] == 2, hotels_merged.columns[[1] + list(range(5, hotels_merged.shape[1]))]]
display(k3)
display(hotels_merged.iloc[k3.index,0].tolist())
k3.shape

,categories,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Hotel,Italian Restaurant,Café,Ice Cream Shop,Plaza,Cocktail Bar,Bed & Breakfast,Restaurant,Church,Dessert Shop,Bookstore
10,Roof Deck,Italian Restaurant,Café,Ice Cream Shop,Plaza,History Museum,Bookstore,Bistro,Church,Pizza Place,Dessert Shop
14,Hotel,Italian Restaurant,Café,Trattoria/Osteria,Pizza Place,Wine Bar,Plaza,French Restaurant,Recording Studio,Restaurant,Pub
15,Hotel,Italian Restaurant,Café,Plaza,Ice Cream Shop,Bed & Breakfast,Bookstore,Dessert Shop,Fountain,Cocktail Bar,Bistro
17,Hotel,Italian Restaurant,Café,Ice Cream Shop,Plaza,Bed & Breakfast,Cocktail Bar,Church,Dessert Shop,Bookstore,Bistro
22,Roof Deck,Italian Restaurant,Cocktail Bar,Trattoria/Osteria,Café,Historic Site,Pizza Place,Wine Bar,French Restaurant,Plaza,Public Art
26,Hotel,Italian Restaurant,Café,Ice Cream Shop,Pizza Place,Bed & Breakfast,Restaurant,Plaza,Bakery,Lounge,Market
31,Hotel,Italian Restaurant,Café,Pizza Place,Bed & Breakfast,Ice Cream Shop,Plaza,Restaurant,Historic Site,Cocktail Bar,Bakery
32,Hotel,Italian Restaurant,Ice Cream Shop,Café,Plaza,Pizza Place,Restaurant,Bed & Breakfast,Supermarket,Pub,Fountain
38,Bed & Breakfast,Italian Restaurant,Café,Pizza Place,Plaza,History Museum,Ice Cream Shop,Paper / Office Supplies Store,Park,Cosmetics Shop,Falafel Restaurant


['Hotel Diana',
 'Hotel Mediterraneo',
 "Hotel Capo D'Africa Rome",
 'Welcome Piram Hotel Rome',
 'Hotel Rex Rome',
 'Hotel Gladiatori Palazzo Manfredi',
 "Hotel D'Este",
 'Hotel Amalfi Rome',
 'Hotel Diocleziano',
 'Hotel Concorde',
 'Edera Hotel',
 'Hotel Tirreno']

(12, 11)

In [60]:
k4=hotels_merged.loc[hotels_merged['Cluster Labels'] == 3, hotels_merged.columns[[1] + list(range(5, hotels_merged.shape[1]))]]
display(k4)
display(hotels_merged.iloc[k4.index,0].tolist())
k4.shape

,categories,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Hotel,Italian Restaurant,Historic Site,Temple,History Museum,Sandwich Place,Monument / Landmark,Pizza Place,Plaza,Ice Cream Shop,Wine Bar
1,Hotel,Italian Restaurant,Historic Site,Ice Cream Shop,Restaurant,Wine Bar,History Museum,Art Museum,Plaza,Pub,Museum
8,Hotel,Italian Restaurant,Historic Site,Temple,Ice Cream Shop,Sandwich Place,Pizza Place,Wine Bar,Monument / Landmark,Plaza,Coffee Shop
30,Hotel,Italian Restaurant,Historic Site,Sandwich Place,Ice Cream Shop,Pizza Place,Wine Bar,Temple,Monument / Landmark,Coffee Shop,Café
35,Hotel,Italian Restaurant,Historic Site,Plaza,Ice Cream Shop,Art Museum,Restaurant,History Museum,Museum,Pub,Monument / Landmark
43,Hotel,Italian Restaurant,Historic Site,Plaza,Wine Bar,History Museum,Art Museum,Temple,Pizza Place,Ice Cream Shop,Monument / Landmark


['Hotel Forum',
 'Hotel Pace Helvezia',
 'Hotel Fori Imperiali Cavalieri',
 'Downtown Accommodation Hotel',
 'Hotel Cosmopolita Rome',
 'Roma Luxus Hotel']

(6, 11)

# 5. Results and Discussion

Our analysis shows that exist 50 hotels registered by Foursquare in Rome in a circular area with a radius of 1.5km, also as we can expect the most common venue in all the Clusters is 'italian Restaurant', so that venue will not necessary in to the recommendation, also we cang see in the first map that the majority of hotels are in the city center, that could be a chracteristic to consider if our client prefer a quite place.Then we can analyze each Cluster and finally get a conclussion.

1)  Cluster 1 contains  16 hotels with 10 tops venues mainly cotains Café/Ice cream shop/wine bar/pizza place...,it lookslike a place where the citizens will enjoy, but maybe not for a tourist who want try and taste differents  things.
2) Cluster 2 contains  16 hotels with 10 tops venues mainly cotains Boutique/Ice cream shop/fountain/shops, it looks like for a tourist who go for shopping and have a good time.
3) Cluster 3 contains  12 hotels with 10 tops venues mainly cotains ice cream shop/plaza/historic sites..., it looks balanced for a tourist who want know about the history, but enjoy his free time too.
4) Cluster 4 contains 6 hotels with 10 tops venues mainly cotains history site/Temples/Museums/Monuments..., it unlike the first cluster looks good for a tourist who want know more about the culture and history of Rome.

# 6. Conlusion

Purpose of this project was to identify a group of hotels for a tourists depending of their pleasures.Target to individuals who want to travel to Rome,Italy for sightseeing.
The Foursquare location data was leveraged to compare each hotel to provide reliable suggestions for individuals who want to choose a hotel according their reasons to travel(sightseeing,business,shopping,etc).With unsupervised learning K-means algorithm, all hotels were clustered in to 4 categories, the advantages of each category was expressed to help individuals choose the ideal group oh hotels to recommend group of hotels to recommend to the client.
This Project simply processed the hotels, and cluster them into 4 categories based one the venues around, the results can only help individuals choose the hotel they want. Further analysis can be done base on these 4 clusters, which can help provide more detail information to clarify the advantages of each.